In [ ]:
!pip install langchain
!pip install sentence-transformers
!pip install faiss-cpu
!pip install boto3
!pip install tqdm

In [ ]:
!pip install -U langchain-community

In [ ]:
!pip install langchain[bedrock]

In [ ]:
!pip install faiss-gpu

In [ ]:
import json
import time
import random
from tqdm import tqdm
from typing import Optional, List
from langchain.embeddings import HuggingFaceEmbeddings
import boto3
import torch
from transformers import AutoTokenizer, AutoModel
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import FAISS
from langchain.embeddings.base import Embeddings
from langchain_core.language_models import BaseLLM
from langchain_core.outputs import Generation, LLMResult
from pydantic import Field


class BedrockLLM(BaseLLM):
    model_id: str = Field(...)
    region: str = Field(...)

    def __init__(self, model_arn: str, region: str = "us-west-2"):
        super().__init__(model_id=model_arn, region=region)
        object.__setattr__(self, "client", boto3.client("bedrock-runtime", region_name=region))

    @property
    def _llm_type(self) -> str:
        return "bedrock-custom"

    def _generate(self, prompts: List[str], stop: Optional[List[str]] = None) -> LLMResult:
        generations = []
        for prompt in prompts:
            for attempt in range(5):
                try:
                    body = {
                        "prompt": prompt,
                        "max_gen_len": 512,
                        "temperature": 0.0,
                        "top_p": 0.9
                    }
                    response = self.client.invoke_model(
                        modelId=self.model_id,
                        body=json.dumps(body).encode("utf-8"),
                        contentType="application/json",
                        accept="application/json"
                    )
                    result = json.loads(response["body"].read())
                    text = result.get("generation") or result.get("outputs", [{}])[0].get("text", str(result))
                    generations.append([Generation(text=text)])
                    break
                except self.client.exceptions.ModelNotReadyException:
                    wait = 2 ** attempt + random.uniform(0, 1)
                    print(f"Model not ready. Retrying in {wait:.1f} seconds...")
                    time.sleep(wait)
                except Exception as e:
                    raise RuntimeError(f"Client error during invoke: {str(e)}")
        return LLMResult(generations=generations)


# Φόρτωση Δεδομένων
with open("sagemaker_ft500vers2 (1) (1).jsonl", "r", encoding="utf-8") as f:
    dataset = [json.loads(line) for line in f]


# Ορισμός Διανυσματικού μοντέλου 
embedding_model = HuggingFaceEmbeddings(model_name="cambridgeltl/SapBERT-from-PubMedBERT-fulltext")

# Φόρτωση Ανακτητή
retriever = FAISS.load_local(
    "sapbert_base_db",
    embedding_model,
    allow_dangerous_deserialization=True
).as_retriever()

# Επιλογή LLM
llm = BedrockLLM(
    model_arn = "meta.llama3-1-70b-instruct-v1:0",
    #model_arn="arn:aws:bedrock:us-west-2:471112783210:imported-model/y9xa363z3o7r",
    #model_arn="arn:aws:bedrock:us-west-2:471112783210:imported-model/1tfsf1vs44wd",
    region="us-west-2"
)

# === Prompt Templates ===
prompt_step1 = PromptTemplate(
    input_variables=["question"],
    template="""
You are a clinical summarization expert. Carefully read the case and provide a concise summary of only the relevant clinical findings, omitting distractors.

CASE AND QUESTION:
{question}

SUMMARY:
"""
)

prompt_step2 = PromptTemplate(
    input_variables=["summary", "context"],
    template="""
You are a senior clinician. Using the summary and current guideline-based knowledge, identify:

1. The most likely diagnosis (**bold**).
2. 2–3 plausible differential diagnoses with one-line justifications.
3. A logical, step-by-step reasoning for your top diagnosis.

CLINICAL SUMMARY:
{summary}

RETRIEVED GUIDELINES:
{context}

DIAGNOSTIC REASONING:
"""
)

prompt_step3 = PromptTemplate(
    input_variables=["summary", "diagnosis", "context"],
    template="""
You are an expert clinician using the VINDICATE + P² mnemonic to evaluate other possible diagnoses.

SUMMARY:
{summary}

INITIAL DIAGNOSIS:
{diagnosis}

RETRIEVED GUIDELINES:
{context}

For each category below, state one plausible alternative diagnosis (or "Not applicable"):

- Vascular, Infectious, Neoplastic, Degenerative, Intoxication/Iatrogenic, Congenital, Autoimmune/Allergic, Trauma, Endocrine/Metabolic, Psychiatric, Paraneoplastic.

Conclude by stating if any alternative is more likely than the initial.

VINDICATE + P² DIFFERENTIAL:
"""
)

prompt_step4 = PromptTemplate(
    input_variables=["summary", "diagnosis", "context"],
    template="""
You are designing a clinical management plan.

SUMMARY:
{summary}

DIAGNOSIS:
{diagnosis}

RETRIEVED GUIDELINES:
{context}

Provide:

1. Short-term treatment steps
2. Long-term management goals
3. Relevant clinical scores or red flags

CLINICAL PLAN:
"""
)

prompt_step5 = PromptTemplate(
    input_variables=["question", "summary", "diagnosis", "vindicate", "management"],
    template="""
You are answering a multiple-choice medical exam question based strictly on the clinical information and reasoning below.

CASE AND QUESTION:
{question}

CLINICAL SUMMARY:
{summary}

DIAGNOSTIC REASONING:
{diagnosis}

DIFFERENTIAL ANALYSIS (VINDICATE):
{vindicate}

CLINICAL MANAGEMENT PLAN:
{management}

INSTRUCTIONS:
- There are exactly 5 answer choices (A–E). Select only one.
- Start your answer with the chosen letter (e.g., "C.") on the first line — no explanation yet.
- Then, write 1–3 concise sentences justifying your choice.
- Briefly reject the other 4 options.
- Use ONLY the above information. Do NOT invent facts or make assumptions.

FORMAT STRICTLY:
<Letter>
<Explanation>
<Why others are incorrect>

FINAL ANSWER:
"""
)


prompt_step6 = PromptTemplate(
    input_variables=["question", "summary", "diagnosis", "management", "answer"],
    template="""
You are a medical board evaluator. Review the following:

QUESTION:
{question}

SUMMARY:
{summary}

DIAGNOSIS:
{diagnosis}

MANAGEMENT:
{management}

FINAL ANSWER:
{answer}

EVALUATE:
- Is the answer choice consistent with the diagnosis and plan?
- Is the reasoning complete and appropriate?
- Are other options addressed clearly?

Respond with only one of:
- VALID: The final answer is correct and well justified.
- INVALID: The answer is inconsistent or the justification is inadequate.
- INCOMPLETE: The reasoning is partially correct but lacks completeness or clarity.

EVALUATION:
"""
)


# αΞΙΟΛΌΓΗΣΗ
output_path = "validation_answers_sapbert_6_steps.jsonl"
with open(output_path, "w", encoding="utf-8") as fout:
    for example in tqdm(dataset):
        question = example["model_input"]

        # Βήμα 1: Περίληψη του κλινικού περιστατικού
        step1_output = LLMChain(llm=llm, prompt=prompt_step1).invoke({
            "question": question
        })["text"]

        # Βήμα 2: Διάγνωση (με χρήση RAG και σχετικών αποσπασμάτων)
        context2 = "\n".join([doc.page_content for doc in retriever.get_relevant_documents(step1_output)[:15]])
        step2_output = LLMChain(llm=llm, prompt=prompt_step2).invoke({
            "summary": step1_output,
            "context": context2
        })["text"]

        # Βήμα 3: VINDICATE – συστηματικός έλεγχος πιθανών αιτιών (με RAG)
        context3 = "\n".join([doc.page_content for doc in retriever.get_relevant_documents(step1_output + step2_output)[:15]])
        step3_output = LLMChain(llm=llm, prompt=prompt_step3).invoke({
            "summary": step1_output,
            "diagnosis": step2_output,
            "context": context3
        })["text"]

        # Βήμα 4: Σχέδιο θεραπείας / διαχείρισης (με RAG και κατευθυντήριες οδηγίες)
        context4 = "\n".join([doc.page_content for doc in retriever.get_relevant_documents(step1_output + step2_output + step3_output)[:15]])
        step4_output = LLMChain(llm=llm, prompt=prompt_step4).invoke({
            "summary": step1_output,
            "diagnosis": step2_output,
            "context": context4
        })["text"]

        # Βήμα 5: Τελική απάντηση (MCQ) χωρίς χρήση RAG
        step5_output = LLMChain(llm=llm, prompt=prompt_step5).invoke({
            "question": question,
            "summary": step1_output,
            "diagnosis": step2_output,
            "vindicate": step3_output,
            "management": step4_output
        })["text"]

        # Βήμα 6: Αξιολόγηση της τελικής απάντησης
        step6_evaluation = LLMChain(llm=llm, prompt=prompt_step6).invoke({
            "question": question,
            "summary": step1_output,
            "diagnosis": step2_output,
            "management": step4_output,
            "answer": step5_output
        })["text"]


        # Εξοδος
        output = {
            "question": question,
            "target_output": example.get("target_output", ""),
            "step1_summary": step1_output,
            "step2_diagnosis": step2_output,
            "step3_vindicate_analysis": step3_output,
            "step4_management": step4_output,
            "step5_final_answer": step5_output,
            "step6_evaluation": step6_evaluation
        }

        fout.write(json.dumps(output) + "\n")
